In [2]:
%matplotlib inline
from mytools3 import read_part_wetzel
import yt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sys,yt
from yt.units import kiloparsec, Msun, km, s
import wutilities as ut

In [51]:
def snapfile_to_readargs(snapfile):
    snapfile = snapfile.rstrip('/')
    if '/' not in snapfile:
        snapfile = './'+snapfile
    from os import path
    if path.isdir(snapfile):
        dodir = True
    else:
        dodir = False

    snapdir,snapshot_name = snapfile.rsplit('/',1)
    snapshot_name,snapnum = snapshot_name.rsplit('_',1)
    snapnum = int(snapnum.split('.')[0])

    if dodir:
        snapshot_name = 'snapshot'      #otherwise, tries to open the folder as a binary

    return snapdir,snapnum,snapshot_name

def read_part_wetzel(fname,types,assign_center=False,subselect=False,**kwargs):
    """
    read particles from a specific filename/snapshot directory.  either do or
    don't assign center.

    subselect is either False/None or a dictionary that contains
        :type:  either "box" or "sphere" -- specifies whether you cut particles within a sphere or particles within a box
        :radius: a number in the units of part that gives either half the box length or the radius (depending on type)
        :center:  a length 3 list-type that gives the center you want to select from.  if not given, then assign_center must be true.
    """

    import gizmo_analysis as gizmo  
    import wutilities as ut
    from os import path
    import numpy as np

    if subselect != None and subselect != False:
        assert type(subselect) == dict
        assert 'type' in subselect.keys()
        assert 'radius' in subselect.keys()
        if assign_center == False:
            assert 'center' in subselect.keys()
            assert type(subselect['center']) == list or type(subselect['center']) == np.ndarray
            assert len(subselect['center']) == 3

    snapdir,snum,snapshot_name = snapfile_to_readargs(fname)

    if path.isfile(fname):
        Read = gizmo.io.ReadClass(snapshot_name_base=snapshot_name.rsplit('_',1)[0])
        part = Read.read_snapshots(types,snapshot_values=snum,snapshot_value_kind='index',snapshot_directory=snapdir,assign_center=assign_center,**kwargs)
    elif path.isdir(fname):
        Read = gizmo.io.ReadClass()
        part = gizmo.io.Read.read_snapshots(types,snapshot_values=snum, snapshot_value_kind='index',snapshot_directory=snapdir,assign_center=assign_center,**kwargs)        
    else:
        raise IOError("{} doesn't exist".format(fname))

    if subselect:
        nkept = 0
        ntossed = 0
        if subselect['type'] == 'box':
            print("Subselecting particles within a cube w/ half a side length of {}".format(subselect['radius']))
        else:
            print("Subselecting particles within a sphere of radius {}".format(subselect['radius']))

        if assign_center:
            cen = part.center_position
        else:
            cen = subselect['center']

        r = subselect['radius']
        for k in part.keys():
            if subselect['type'] == 'box':
                msk = boxmsk(part[k]['position'],cen,r)
            else:
                msk = fast_dist(part[k]['position'],cen) <= r

            tn = np.count_nonzero(msk)
            nkept += tn
            ntossed += msk.shape[0] - tn

            for prop in part[k].keys():
                part[k][prop] = part[k][prop][msk]

    if 'force_float32' in kwargs:
        #transform the mass back into float64 cause those have to be summed and that can cause problems
        for ptype in part:
            if 'mass' in part[ptype]:
                part[ptype]['mass'] = part[ptype]['mass'].astype(np.float64)
    return part

def boxmsk(pos,cen,size):
    """
    returns a binary array that identifies points
    that fall within a box from -size to +size in
    each dimension

    :param pos:
        the points (3xN) to cut
    :param cen:
        the center of the box
    :param size:
        half the extent of the box
    """

    from numpy import array,abs
    px,py,pz = pos[:,0],pos[:,1],pos[:,2]
    x,y,z = cen
    return (abs(px-x)<size)&(abs(py-y)<size)&(abs(pz-z)<size)

In [30]:
part = read_part_wetzel('output/snapdir_600/', ['star', 'gas'], properties=['position','velocity','mass'], assign_center=True)


# in wutilities.simulation.Snapshot():
  read snapshot_times.txt
  reading snapshot index = 600, redshift = 0.000

# in gizmo_analysis.gizmo_io.Read():
* read header from: output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
  dark      (id = 1): 70514272 particles
  dark.2    (id = 2): 5513331 particles
  gas       (id = 0): 57060074 particles
  star      (id = 4): 13976485 particles
  blackhole (id = 5): 0 particles

* read particles
  read star  : ['mass', 'position', 'velocity']
  read gas   : ['mass', 'position', 'velocity']
  from: snapshot_600.0.hdf5
  from: snapshot_600.1.hdf5
  from: snapshot_600.2.hdf5
  from: snapshot_600.3.hdf5

* read cosmological parameters from: initial_conditions/ic_agora_m12i.conf

* checking sanity of particle properties

* assigning center of galaxy/halo:
  position = (41792.147, 44131.235, 46267.679) [kpc comoving]
  velocity = (-52.5, 71.9, 95.2) [km / s]



In [8]:
cen = part.center_position
vel = part.center_velocity

In [44]:
bsize = 30
bl = [-bsize/2, bsize/2]
bbox = np.array( [ bl, bl, bl])

data = {}
for ptype in part:
    msk = boxmsk(part[ptype]['position'], cen, bsize/2)

    data[(ptype,'particle_mass')] = part[ptype]['mass'][msk]

    data[(ptype,'particle_position_x')] = part[ptype]['position'][msk][:,0] - cen[0]
    data[(ptype,'particle_position_y')] = part[ptype]['position'][msk][:,1] - cen[1]
    data[(ptype,'particle_position_z')] = part[ptype]['position'][msk][:,2] - cen[2]

    data[(ptype,'particle_velocity_x')] = part[ptype]['velocity'][msk][:,0] - vel[0]
    data[(ptype,'particle_velocity_y')] = part[ptype]['velocity'][msk][:,1] - vel[1]
    data[(ptype,'particle_velocity_z')] = part[ptype]['velocity'][msk][:,2] - vel[2]


ds = yt.load_particles(data, length_unit=kiloparsec, mass_unit=Msun, n_ref=32, bbox=bbox, velocity_unit=km/s)


yt : [INFO     ] 2018-05-25 14:46:30,928 Parameters: current_time              = 0.0
yt : [INFO     ] 2018-05-25 14:46:30,929 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2018-05-25 14:46:30,930 Parameters: domain_left_edge          = [-15. -15. -15.]
yt : [INFO     ] 2018-05-25 14:46:30,931 Parameters: domain_right_edge         = [15. 15. 15.]
yt : [INFO     ] 2018-05-25 14:46:30,931 Parameters: cosmological_simulation   = 0.0


In [52]:
ds.derived_field_list

[('all', 'mesh_id'),
 ('all', 'particle_angular_momentum'),
 ('all', 'particle_angular_momentum_magnitude'),
 ('all', 'particle_angular_momentum_x'),
 ('all', 'particle_angular_momentum_y'),
 ('all', 'particle_angular_momentum_z'),
 ('all', 'particle_cylindrical_velocity_theta'),
 ('all', 'particle_cylindrical_velocity_z'),
 ('all', 'particle_mass'),
 ('all', 'particle_ones'),
 ('all', 'particle_position'),
 ('all', 'particle_position_cylindrical_radius'),
 ('all', 'particle_position_cylindrical_theta'),
 ('all', 'particle_position_cylindrical_z'),
 ('all', 'particle_position_relative'),
 ('all', 'particle_position_relative_x'),
 ('all', 'particle_position_relative_y'),
 ('all', 'particle_position_relative_z'),
 ('all', 'particle_position_spherical_phi'),
 ('all', 'particle_position_spherical_radius'),
 ('all', 'particle_position_spherical_theta'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_radial_velocity'),
 (

In [49]:
prj = yt.ProjectionPlot(ds, 0, ('deposit', 'gas_nn_velocity_x'), center=[0,0,0], width=(bsize, 'kpc'))

yt : [INFO     ] 2018-05-25 14:48:02,641 Projection completed
yt : [INFO     ] 2018-05-25 14:48:02,641 xlim = -15.000000 15.000000
yt : [INFO     ] 2018-05-25 14:48:02,642 ylim = -15.000000 15.000000
yt : [INFO     ] 2018-05-25 14:48:02,643 xlim = -15.000000 15.000000
yt : [INFO     ] 2018-05-25 14:48:02,643 ylim = -15.000000 15.000000
yt : [INFO     ] 2018-05-25 14:48:02,644 Making a fixed resolution buffer of (('deposit', 'gas_nn_velocity_x')) 800 by 800


In [50]:
# prj.set_unit('gas_cic_velocity_x','km/s')
prj.save()

yt : [INFO     ] 2018-05-25 14:48:04,006 Saving plot ParticleData_Projection_x_gas_nn_velocity_x.png


['ParticleData_Projection_x_gas_nn_velocity_x.png']